# Import Libraries

In [1]:
import mysql.connector

In [297]:
import pickle
import ruptures as rpt
from prettytable import PrettyTable

# Connect to MySQL Database

In [2]:
mydb = mysql.connector.connect(
    host='localhost',
    user='root',
    passwd='slow675',
    database='amd_project'
    )

In [3]:
print(mydb)

In [4]:
mycursor = mydb.cursor()

# Load Dataset

In [5]:
%run ../ddc/load_dataset.ipynb

In [284]:
subj_range = np.hstack((np.arange(2001,2002),np.arange(3001,3006)))

all_patients = [str(i) for i in subj_range]

In [285]:
all_patients

['2001', '3001', '3002', '3003', '3004', '3005']

In [7]:
X_all_p, y_all_p, subj_all_p, ts_all_p, hr_all_p = load_all_data(all_patients)

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished loading


In [152]:
mypath = '../../DDC_Data/'
basepath = '../../'

df_all_p = pd.DataFrame()

for subject_id in all_patients:
    print("Loading {0}'s data".format(subject_id))
    
    acc_filepath = mypath + subject_id + '/' + subject_id + '-log_acc.csv'
    df_raw = pd.read_csv(acc_filepath, header=None, names=['x','y','z','timestamp'])

    df_timer, rec_date, start_time, end_time = load_timer(subject_id)
    df_filt = load_acc(subject_id, rec_date, start_time, end_time)
    df_hr = load_hr(subject_id, rec_date, start_time, end_time)

    df1 = merge_acc_and_hr(df_filt, df_hr)
    
    cols = ['x','y','z']
    xyz_ = df1[cols].to_dict('split')['data']
    xyz_new = MinMaxScaler().fit_transform(xyz_)
    
    for i in range(len(cols)):
        df1[cols[i]] = pd.Series(xyz_new.transpose()[i])

    X_i_p = np.array(df1[cols].to_dict(orient='split')['data'])
    subj_all_p = np.array([subject_id for i in range(X_all_p.shape[0])])
    
    df_all_p = df_all_p.append(df1, sort=False)
    
print('Finished Loading')

Loading 2001's data
Loading 3001's data
Loading 3002's data
Loading 3003's data
Loading 3004's data
Loading 3005's data
Finished Loading


In [153]:
df_all_p['label'] = [None for i in range(df_all_p.shape[0])]
df_all_p_sorted = df_all_p.copy()
df_all_p_sorted = df_all_p_sorted.reset_index(drop=True)

In [154]:
df_all_p_sorted

,ID,timestamp,x,y,z,HR,label
0,2001,2019-01-16 13:58:47.956,0.246041,0.475436,0.576183,68.205284,None
1,2001,2019-01-16 13:58:48.117,0.246431,0.473881,0.576429,68.205284,None
2,2001,2019-01-16 13:58:48.278,0.246236,0.474464,0.573722,68.205284,None
3,2001,2019-01-16 13:58:48.439,0.246041,0.473881,0.574460,67.365480,None
4,2001,2019-01-16 13:58:48.6,0.246821,0.472521,0.575444,67.365480,None
5,2001,2019-01-16 13:58:48.76,0.247016,0.474659,0.572246,67.365480,None
6,2001,2019-01-16 13:58:48.922,0.247211,0.477574,0.573722,67.365480,None
7,2001,2019-01-16 13:58:49.08,0.245262,0.469994,0.575198,67.365480,None
8,2001,2019-01-16 13:58:49.243,0.246041,0.475047,0.571754,67.365480,None
9,2001,2019-01-16 13:58:49.402,0.246236,0.471937,0.570277,67.365480,None


In [155]:
df_all_p_sorted['Milliseconds'] = df_all_p_sorted['timestamp'].apply(lambda x: x.split(' ')[1].split('.')[1])

# Predict Labels

In [156]:
filename = basepath + 'model/knn_model_patients.pkl'

model = pickle.load(open(filename,'rb'))

In [245]:
%run ../ddc/preprocessing.ipynb

In [158]:
cols = ['x','y','z']

X_all_p = np.array(df_all_p_sorted[cols].to_dict('split')['data'])
y_all_p = np.zeros(X_all_p.shape[0])

In [159]:
# cols = ['x','y','z']

# model_alg = "l1"
# algo = rpt.Pelt(model=model_alg, min_size=3, jump=5).fit(X_all_p)

# bkps = [100,200,300]
# my_bkps = algo.predict(pen=50)

# # show results
# fig, ax = rpt.display(X_all_p, bkps, my_bkps, figsize=(10, 6))
# plt.show()

In [160]:
y_pred_all = []
df_grouped = df_all_p_sorted.groupby('ID')

for x in df_grouped:
    label_grp = x[0]
    
    df_grp = df_grouped.get_group(label_grp)
    print(np.max(df_grp['x']))
    
    cols = ['x','y','z']

    X_all_p = np.array(df_grp[cols].to_dict('split')['data'])
    y_all_p = np.zeros(X_all_p.shape[0])

    X_impure, y_impure = prepare_impure_label(X_all_p, y_all_p)
    y_pred = model.predict(X_impure)

    window_length = 60
    y_pred_fill = np.hstack(([y_pred[0] for i in range(window_length-1)], y_pred))
    
    y_pred_all.append(y_pred_fill)

1.0
1.0
1.0
1.0
1.0
1.0


In [161]:
y_pred_all = np.hstack(y_pred_all)

In [162]:
df_all_p_sorted['y_pred'] = pd.Series(y_pred_all)
df_all_p_sorted['label'] = pd.Series([label_list[i] for i in y_pred_all])

In [164]:
y_pred_all.shape, df_all_p_sorted.shape

((16578,), (16578, 9))

# Analyze Predicted Results

In [306]:
%run mysql_connector_functions.ipynb

In [166]:
df_date = df_all_p_sorted.copy()
df_date['date'] = df_date['timestamp'].apply(lambda x: x.split(' ')[0])
df_date['time'] = df_date['timestamp'].apply(lambda x: x.split(' ')[1])

cols = ['ID','date','time','Milliseconds','x','y','z','HR','y_pred','label']
df_date = df_date[cols]

In [169]:
df_date

,ID,date,time,Milliseconds,x,y,z,HR,y_pred,label
0,2001,2019-01-16,13:58:47.956,956,0.246041,0.475436,0.576183,68.205284,2,stand
1,2001,2019-01-16,13:58:48.117,117,0.246431,0.473881,0.576429,68.205284,2,stand
2,2001,2019-01-16,13:58:48.278,278,0.246236,0.474464,0.573722,68.205284,2,stand
3,2001,2019-01-16,13:58:48.439,439,0.246041,0.473881,0.574460,67.365480,2,stand
4,2001,2019-01-16,13:58:48.6,6,0.246821,0.472521,0.575444,67.365480,2,stand
5,2001,2019-01-16,13:58:48.76,76,0.247016,0.474659,0.572246,67.365480,2,stand
6,2001,2019-01-16,13:58:48.922,922,0.247211,0.477574,0.573722,67.365480,2,stand
7,2001,2019-01-16,13:58:49.08,08,0.245262,0.469994,0.575198,67.365480,2,stand
8,2001,2019-01-16,13:58:49.243,243,0.246041,0.475047,0.571754,67.365480,2,stand
9,2001,2019-01-16,13:58:49.402,402,0.246236,0.471937,0.570277,67.365480,2,stand


## All Day Summary

In [307]:
df_summary_all = get_df_summary_all(all_patients, df_date)

In [312]:
df_summary_all = df_summary_all.reset_index(drop=True)
df_summary_all

,ID,date,from,to,sit,sleep,stand,walk
0,2001,2019-01-16,13:0:0,14:0:0,0:0:35,0:0:00,0:0:36,0:0:00
1,2001,2019-01-16,14:0:0,15:0:0,0:1:07,0:1:10,0:4:03,0:3:04
2,3001,2019-03-05,10:0:0,11:0:0,0:1:55,0:1:09,0:3:02,0:1:04
3,3002,2019-03-05,12:0:0,13:0:0,0:1:06,0:1:10,0:2:48,0:0:30
4,3003,2019-03-05,12:0:0,13:0:0,0:1:28,0:1:15,0:2:09,0:1:27
5,3004,2019-03-05,12:0:0,13:0:0,0:1:53,0:1:18,0:2:44,0:0:56
6,3005,2019-03-05,12:0:0,13:0:0,0:0:01,0:0:00,0:0:30,0:0:00
7,3005,2019-03-05,13:0:0,14:0:0,0:1:44,0:1:17,0:1:47,0:0:59


## Activity Analysis

In [293]:
subject_id = '2001'
label_period, period_list, df_grp_date = get_period_for_each_label(subject_id, df_date)

2019-01-16


In [300]:
print(len(label_period))

label_cnt_list = [0 for i in range(len(label_list))]
for lb_p in label_period:
    label_i = lb_p[lb_idx]
    
    label_cnt_list[label_i] += 1

203


In [301]:
label_cnt_list

[17, 6, 89, 91]

In [302]:
total_secs = []
for i in range(len(period_list)):    
    secs = 0
    for p_i in period_list[i]:
        sec = calc_sec(p_i[2]) - calc_sec(p_i[1])
        secs += sec
    
    secs = round(secs, 3)
    total_secs.append(secs)
    
percent_secs = [round(t/np.sum(total_secs)*100, 3) for t in total_secs]

tb = PrettyTable(['Label', 'Minutes', 'Percentage', 'Activity Count'])

for i in range(len(LABELS)):
    tb.add_row([label_list[i], convert_time_to_string(total_secs[i]), percent_secs[i], label_cnt_list[i]])

tb.add_row(['', '', '',''])
tb.add_row(['total', convert_time_to_string(round(np.sum(total_secs), 3)), 
            round(np.sum(percent_secs), 3), len(label_period)])

In [303]:
print(tb)

+-------+---------+------------+----------------+
| Label | Minutes | Percentage | Activity Count |
+-------+---------+------------+----------------+
|  sit  |  0:1:43 |   16.17    |       17       |
| sleep |  0:1:10 |   11.011   |       6        |
| stand |  0:4:39 |   43.849   |       89       |
|  walk |  0:3:04 |   28.97    |       91       |
|       |         |            |                |
| total | 0:10:37 |   100.0    |      203       |
+-------+---------+------------+----------------+


# Insert to Database

In [240]:
sql = "INSERT INTO Patient (ID, Dateandtime, Milliseconds, X, Y, Z, HR, Label) \
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_all_p_sorted['ID'], 
               df_all_p_sorted['timestamp'], 
               df_all_p_sorted['Milliseconds'],
               df_all_p_sorted['x'], 
               df_all_p_sorted['y'], 
               df_all_p_sorted['z'], 
               df_all_p_sorted['HR'], 
               df_all_p_sorted['label']):
    
    mycursor.execute(sql, row)

mydb.commit()

In [314]:
sql = "INSERT INTO AllDaySummary (ID, Date, TimeFrom, TimeUntil, DurationSit, DurationSleep, DurationStand, DurationWalk)\
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)"

for row in zip(df_summary_all['ID'], 
               df_summary_all['date'], 
               df_summary_all['from'],
               df_summary_all['to'], 
               df_summary_all['sit'], 
               df_summary_all['sleep'], 
               df_summary_all['stand'], 
               df_summary_all['walk']):
    
    mycursor.execute(sql, row)
    
mydb.commit()

# Show Results

In [242]:
mycursor.execute("SELECT * FROM Patient")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(50):
    print(myresult[x])
    

16578
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '117', 0.2464, 0.4739, 0.5764, 68.2053, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '278', 0.2462, 0.4745, 0.5737, 68.2053, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '439', 0.246, 0.4739, 0.5745, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 48), '956', 0.246, 0.4754, 0.5762, 68.2053, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '08', 0.2453, 0.47, 0.5752, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '243', 0.246, 0.475, 0.5718, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '402', 0.2462, 0.4719, 0.5703, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '6', 0.2468, 0.4725, 0.5754, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '76', 0.247, 0.4747, 0.5722, 67.3655, 'stand')
(2001, datetime.datetime(2019, 1, 16, 13, 58, 49), '922', 0.2472, 0.4776, 0.5737, 67.3655, 'stand')
(2001

In [315]:
mycursor.execute("SELECT * FROM AllDaySummary")

myresult = mycursor.fetchall()

print(len(myresult))

for x in range(len(myresult)):
    print(myresult[x])

8
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 46800), datetime.timedelta(0, 50400), datetime.timedelta(0, 35), datetime.timedelta(0), datetime.timedelta(0, 36), datetime.timedelta(0))
(2001, datetime.date(2019, 1, 16), datetime.timedelta(0, 50400), datetime.timedelta(0, 54000), datetime.timedelta(0, 67), datetime.timedelta(0, 70), datetime.timedelta(0, 243), datetime.timedelta(0, 184))
(3001, datetime.date(2019, 3, 5), datetime.timedelta(0, 36000), datetime.timedelta(0, 39600), datetime.timedelta(0, 115), datetime.timedelta(0, 69), datetime.timedelta(0, 182), datetime.timedelta(0, 64))
(3002, datetime.date(2019, 3, 5), datetime.timedelta(0, 43200), datetime.timedelta(0, 46800), datetime.timedelta(0, 66), datetime.timedelta(0, 70), datetime.timedelta(0, 168), datetime.timedelta(0, 30))
(3003, datetime.date(2019, 3, 5), datetime.timedelta(0, 43200), datetime.timedelta(0, 46800), datetime.timedelta(0, 88), datetime.timedelta(0, 75), datetime.timedelta(0, 129), datetime.timede

In [18]:
sql = "DELETE FROM Patient"

mycursor.execute(sql)

mydb.commit()

In [313]:
mycursor.execute("DROP TABLE AllDaySummary")
# mycursor.execute("DROP TABLE ActivityAnalysis")
# mycursor.execute("DROP TABLE Patient")